## Drug Review Sentiment Classifier - LLM

In [ ]:
# 1. Mount Google Drive (optional but recommended for saving models/results)
from google.colab import drive
drive.mount('/content/drive')

# 2. Clone your GitHub repo
!git clone https://github.com/sandragodinhosilva/drug-review-sentiment
%cd drug-review-sentiment

# 3. Install dependencies
%pip install -r requirements.txt

# 4. (Optional) Verify GPU availability
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


In [ ]:
# =============================
# 1. Imports
# =============================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import mlflow
import optuna

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate

from transformers import TrainingArguments
import inspect


/home/sandra/drug-review-sentiment/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# =============================
# 1. Load data
# =============================

train = pd.read_csv(os.path.join("..", "data", "drugsComTrain_raw.tsv"), sep="\t")
test = pd.read_csv(os.path.join("..", "data", "drugsComTest_raw.tsv"), sep="\t")


def create_label(rating):
    """
    Convert numeric rating (1 - 10) into binary label: 0 = negative, 1 = positive.
    """
    return 1 if rating > 5 else 0

train["sentiment"] = train["rating"].apply(create_label)
test["sentiment"]  = test["rating"].apply(create_label)

# Keep only required columns
train = train[["review", "sentiment"]]
test  = test[["review", "sentiment"]]

# Convert to Hugging Face datasets
train_ds = Dataset.from_pandas(train)
test_ds  = Dataset.from_pandas(test)

# Create DatasetDict
dataset = DatasetDict({
    "train": train_ds,
    "test": test_ds
})


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 53766
    })
})


In [4]:
model_ckpt = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Preprocess Data
def tokenize(batch):
    return tokenizer(batch["review"], truncation=True, padding="max_length", max_length=256)

tokenized = dataset.map(tokenize, batched=True)
tokenized = tokenized.remove_columns(["review"])
tokenized = tokenized.rename_column("sentiment", "labels")
tokenized.set_format("torch")


Map: 100%|██████████| 53766/53766 [00:08<00:00, 6142.24 examples/s]


In [9]:
# Training Setup
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }

training_args = TrainingArguments(
    output_dir="results/biobert",

    # Defines when to run evaluation (trainer.evaluate).   
    # "epoch" → run evaluation at the end of every epoch. Alternatives: "steps" (every N steps) or "no".
    eval_strategy="epoch", 

    save_strategy="epoch", # when to save checkpoints
    learning_rate=2e-5, # Initial learning rate for AdamW optimizer.
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # Number of passes over the entire training dataset.
    weight_decay=0.01, # Regularization: penalizes large weights in the optimizer.
    load_best_model_at_end=True, # After training, reloads the best checkpoint (based on evaluation metric).
    logging_dir="logs", # Directory for logs (can be used with TensorBoard).
)

In [ ]:
# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluate
results = trainer.evaluate()
print(results)

/home/sandra/drug-review-sentiment/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 